# Thermostat demo

In [1]:
!pip3 install --upgrade pip
!pip3 install cmake
!pip3 install cython
!pip3 install numpy
!pip3 install torch
!pip3 install datasets
!pip3 install spacy
!pip3 install sentencepiece
!pip3 install transformers
!pip3 install overrides
!pip3 install jsonnet
!pip3 install sklearn
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import warnings
# Suppress warnings
warnings.filterwarnings('ignore')

In [3]:
import sys
# Include root directory in module path
sys.path.append('src')

from pprint import pprint

import thermostat  # Accompanying library

# Load dataset

This will use the dataset script ("thermostat.py") in the "thermostat" directory.
In this example, we use the test set of the "imdb-bert-lig" configuration.
This refers to Layer Integrated Gradients (LIG) explanations of the predictions by a BERT model that has been fine-tuned on the IMDb (train) dataset and evaluated on the IMDb test dataset.
In other words, we load the 25k test examples from the IMDb test plus the BERT predictions and the feature attributions from a Layer Integrated Gradients explainer.

In [51]:
data = thermostat.load("imdb-bert-lig")
print(data)

Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/imdb-bert-lig/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: imdb-bert-lig
IMDb dataset, BERT model, Layer Integrated Gradients explanations
Explainer: LayerIntegratedGradients
Model: textattack/bert-base-uncased-imdb
Dataset: imdb



Now let's inspect a single instance of the loaded dataset.
Here, we will stick to the functionality that the datasets library already supplies us with.
For readability purposes, we will not print the whole content of that instance.
Instead, we're showing only the first few entries of the attributions and the input_ids, respectively.

In [52]:
example = data[0]

print(f'Index: {example.idx}')
print(f'Attributions (first 4): {example.attributions[:4]}')
print(f'True label: {example.true_label}')
print(f'Predicted label: {example.predicted_label}')

Index: 0
Attributions (first 4): [0.0, -0.18146862089633942, 0.13141055405139923, 0.2554049491882324]
True label: pos
Predicted label: pos


In [53]:
from pprint import pprint

pprint(example.explanation)

[('[CLS]', 0.0, 0),
 ('i', -0.18146862089633942, 1),
 ('went', 0.13141055405139923, 2),
 ('and', 0.2554049491882324, 3),
 ('saw', 0.05605292320251465, 4),
 ('this', -0.045899324119091034, 5),
 ('movie', 0.8242700695991516, 6),
 ('last', 0.37140950560569763, 7),
 ('night', 0.2645639479160309, 8),
 ('after', -0.06957561522722244, 9),
 ('being', -0.01598694920539856, 10),
 ('coa', 0.01185671053826809, 11),
 ('##xed', 0.02574639953672886, 12),
 ('to', 0.041683878749608994, 13),
 ('by', -0.03300376608967781, 14),
 ('a', 0.011767166666686535, 15),
 ('few', 0.033511724323034286, 16),
 ('friends', 0.08964859694242477, 17),
 ('of', -0.006024990230798721, 18),
 ('mine', -0.008424906991422176, 19),
 ('.', 0.013937687501311302, 20),
 ('i', -0.031755100935697556, 21),
 ("'", 0.024647776037454605, 22),
 ('ll', -0.0022892060223966837, 23),
 ('admit', -0.02394527569413185, 24),
 ('that', 0.02765575423836708, 25),
 ('i', -0.07030332088470459, 26),
 ('was', -0.05606917291879654, 27),
 ('reluctant', 0.08

In [54]:
import pandas as pd
pd.set_option('display.max_columns', None)

print(example.heatmap)

token_index    0         1          2         3          4         5    \
token        [CLS]         i       went       and        saw      this   
attribution      0 -0.117371  0.0849944  0.165192  0.0362542 -0.029687   
text_field    text      text       text      text       text      text   

token_index       6         7         8          9          10         11   \
token           movie      last     night      after      being     coaxed   
attribution  0.533126  0.240222  0.171116 -0.0450005 -0.0103401  0.0166524   
text_field       text      text      text       text       text       text   

token_index        13         14          15         16         17   \
token               to         by           a        few    friends   
attribution  0.0269605 -0.0213463  0.00761083  0.0216749  0.0579834   
text_field        text       text        text       text       text   

token_index         18         19          20         21         22   \
token                of       min

In [57]:
example.fill_text_fields()
print(' '.join(example.text['token']))

i went and saw this movie last night after being coaxed to by a few friends of mine . i ' ll admit that i was reluctant to see it because from what i knew of ashton kutcher he was only able to do comedy . i was wrong . kutcher played the character of jake fischer very well , and kevin costner played ben randall with such professionalism . the sign of a good movie is that it can toy with our emotions . this one did exactly that . the entire theater ( which was sold out ) was overcome by laughter during the first half of the movie , and were moved to tears during the second half . while exiting the theater i not only saw many women in tears , but many full grown men as well , trying desperately not to let anyone see them crying . this movie was great , and i suggest that you go see it before you judge .


# Visualize data
Can we make this a bit more readable?  
Of course!  


In [7]:
example.render()

# Aggregate data
Let us first compare the heatmaps of two different models on the same dataset+explainer configuration, MNLI + Occlusion.

In [3]:
bert = thermostat.load("multi_nli-bert-occ")
electra = thermostat.load("multi_nli-electra-occ")

Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-bert-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: multi_nli-bert-occ


Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-electra-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: multi_nli-electra-occ


In [4]:
for model_name, data in zip(["bert", "electra"], [bert, electra]):
    print(model_name)
    data.classification_report()
    print('=====================\n\n')

Decoding Thermounits:   0%|          | 0/9815 [00:00<?, ?it/s]

bert
The instances (Thermounits) of this Thermopack are placeholders and have to be fully processed before all attributes and metadata are available.


Decoding Thermounits:   0%|          | 0/9815 [00:00<?, ?it/s]

               precision    recall  f1-score   support

contradiction       0.86      0.87      0.86      3213
   entailment       0.89      0.85      0.87      3479
      neutral       0.79      0.82      0.81      3123

     accuracy                           0.85      9815
    macro avg       0.85      0.85      0.85      9815
 weighted avg       0.85      0.85      0.85      9815



electra
The instances (Thermounits) of this Thermopack are placeholders and have to be fully processed before all attributes and metadata are available.


Decoding Thermounits: 100%|██████████| 9815/9815 [00:03<00:00, 3260.44it/s]

               precision    recall  f1-score   support

   entailment       0.93      0.87      0.90      3479
      neutral       0.83      0.87      0.85      3123
contradiction       0.91      0.92      0.91      3213

     accuracy                           0.89      9815
    macro avg       0.89      0.89      0.89      9815
 weighted avg       0.89      0.89      0.89      9815





In [4]:
disagreement = [(b, e) for (b, e) in zip(bert, electra) if b.predicted_label != e.predicted_label]
contradictions = [(i, unit)[0] for i, unit in enumerate(disagreement) if unit[0].true_label == 'contradiction']

In [43]:
# good examples in disagreement: 16
index = contradictions[16]
u_b, u_e = disagreement[index]
print(f'Instance index: {u_b.idx}')

print(f'Model: {u_b.model_name} | Pred: {u_b.predicted_label} | True: {u_b.true_label}')
u_b.render()
print(f'Model: {u_e.model_name} | Pred: {u_e.predicted_label} | True: {u_e.true_label}')
u_e.render()

Instance index: 378
Model: textattack/bert-base-uncased-MNLI | Pred: entailment | True: contradiction


Model: howey/electra-base-mnli | Pred: contradiction | True: contradiction


# Explainer comparison

In [46]:
unit_index = 378
u_occ = thermostat.load("multi_nli-bert-occ")[unit_index]
u_intg = thermostat.load("multi_nli-bert-lig")[unit_index]
u_lime = thermostat.load("multi_nli-bert-lime")[unit_index]

Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-bert-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: multi_nli-bert-occ


Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-bert-lig/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: multi_nli-bert-lig


Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-bert-lime/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: multi_nli-bert-lime


In [49]:
print(f'{u_occ.explainer_name} --- same map as in previous example')
u_occ.render()
print(u_intg.explainer_name)
u_intg.render()
print(u_lime.explainer_name)
u_lime.render()

Occlusion --- same map as in previous example


LayerIntegratedGradients


LimeBase


# Rank correlation
We can also investigate how well two explainers on the same dataset+model combination correlate regarding their attribution scores.

In [4]:
import numpy as np
from scipy.stats import spearmanr, kendalltau

imdb_lime = thermostat.load("imdb-bert-lime")[:500]
imdb_lime = np.array(imdb_lime["attributions"]).flatten()
imdb_lime = imdb_lime / np.linalg.norm(imdb_lime)

imdb_intg = thermostat.load("imdb-bert-lig")[:500]
imdb_intg = np.array(imdb_intg["attributions"]).flatten()
imdb_intg = imdb_intg / np.linalg.norm(imdb_intg)

spearman_imdb = spearmanr(imdb_lime, imdb_intg)
kendall_imdb = kendalltau(imdb_lime, imdb_intg)
print(spearman_imdb, kendall_imdb)

Loading Thermostat configuration: imdb-bert-lime


Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/imdb-bert-lime/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)
Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/imdb-bert-lig/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: imdb-bert-lig
SpearmanrResult(correlation=0.03625706244605674, pvalue=3.2572266817386955e-75) KendalltauResult(correlation=0.027750187009839406, pvalue=5.386808373974137e-74)


In [5]:
mnli_lime = thermostat.load("multi_nli-bert-lime")[:500]
mnli_lime = np.array(mnli_lime["attributions"]).flatten()
mnli_lime = mnli_lime / np.linalg.norm(mnli_lime)

mnli_intg = thermostat.load("multi_nli-bert-lig")[:500]
mnli_intg = np.array(mnli_intg["attributions"]).flatten()
mnli_intg = mnli_intg / np.linalg.norm(mnli_intg)

spearman_mnli = spearmanr(mnli_lime, mnli_intg)
kendall_mnli = kendalltau(mnli_lime, mnli_intg)
print(spearman_mnli, kendall_mnli)

Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-bert-lime/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: multi_nli-bert-lime


Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-bert-lig/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: multi_nli-bert-lig
SpearmanrResult(correlation=0.10676521334684468, pvalue=0.0) KendalltauResult(correlation=0.10325044792397908, pvalue=0.0)
